# Sistem Pengenalan Plat Nomor Kendaraan Jakarta
## Analisis CCTV Jalan Raya dengan Deep Learning

**Proyek**: Deteksi dan ekstraksi plat nomor dari gambar CCTV jalan Jakarta  
**Dataset**: Koleksi gambar kendaraan dari berbagai sudut CCTV di Jakarta  
**Kondisi**: Variasi cuaca (siang/malam, hujan/cerah) dan pencahayaan  

---

In [ ]:
# Section 1: Instalasi Dependencies
# Tujuan: Menginstall library yang diperlukan untuk computer vision dan deep learning

# Instalasi package untuk deteksi objek dan OCR
# ultralytics: YOLOv8 untuk deteksi kendaraan
# easyocr: Ekstraksi teks dari plat nomor
!pip install ultralytics easyocr opencv-python matplotlib numpy

print("Instalasi dependencies berhasil!")

## Section 2: Konfigurasi Environment
**Tujuan**: Setup environment dan path dataset yang fleksibel

### 2A. Setup Google Colab (Opsional)
**Catatan**: Uncomment jika menggunakan Google Colab

In [ ]:
# 2A. Setup untuk Google Colab (uncomment jika diperlukan)
# from google.colab import drive
# drive.mount('/content/drive')
# print("Google Drive berhasil di-mount!")

print("Google Colab setup di-skip (uncomment jika diperlukan)")

In [ ]:
# 2B. Konfigurasi Path Dataset
# Tujuan: Menentukan lokasi dataset gambar CCTV Jakarta

import os

# Konfigurasi path dataset - sesuaikan dengan environment Anda
# Opsi 1: Google Colab
# data_folder = '/content/drive/MyDrive/Computer Vision/plat_detection/7.TL.Senen'

# Opsi 2: Local development
data_folder = './dataset/jakarta_cctv'  # Sesuaikan dengan path lokal Anda

# Opsi 3: Custom path
# data_folder = input("Masukkan path folder dataset: ")

# Validasi path dataset
if os.path.exists(data_folder):
    img_count = len([f for f in os.listdir(data_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Dataset ditemukan: {data_folder}")
    print(f"Total gambar: {img_count} files")
else:
    print(f"Path dataset tidak ditemukan: {data_folder}")
    print("Silakan periksa path atau buat folder dataset terlebih dahulu")

## Section 3: Import Libraries & Inisialisasi Model
**Tujuan**: Memuat semua library yang diperlukan dan inisialisasi model AI

### 3A. Import Libraries

In [ ]:
# 3A. Import library untuk computer vision dan deep learning
from ultralytics import YOLO          # YOLOv8 untuk deteksi kendaraan
import easyocr                        # OCR untuk membaca teks plat nomor
import cv2                           # OpenCV untuk image processing
import numpy as np                   # Operasi array dan matematika
import matplotlib.pyplot as plt      # Visualisasi dan plotting
import os                           # Operasi file dan folder
from typing import List, Tuple      # Type hints untuk code quality

print("Import libraries berhasil!")
print("OpenCV version:", cv2.__version__)

In [ ]:
# 3B. Inisialisasi Model AI
# Inisialisasi model YOLO untuk deteksi kendaraan
print("Menginisialisasi model YOLO...")
model = YOLO('yolov8n.pt')  # Model nano untuk performa optimal
print("Model YOLO berhasil dimuat!")

# Inisialisasi EasyOCR dengan dukungan bahasa Indonesia dan Inggris
print("Menginisialisasi EasyOCR...")
try:
    # Deteksi GPU untuk performa optimal
    import torch
    gpu_available = torch.cuda.is_available()
    print(f"GPU tersedia: {gpu_available}")
    
    reader = easyocr.Reader(['en'], gpu=gpu_available)  # Bahasa Inggris untuk plat Indonesia
    print("EasyOCR berhasil dimuat!")
except Exception as e:
    print(f"Warning: {e}")
    reader = easyocr.Reader(['en'], gpu=False)  # Fallback ke CPU
    print("EasyOCR dimuat dengan CPU")

print("Setup model selesai! Siap memproses gambar CCTV.")

## Section 4: Image Enhancement Functions
**Tujuan**: Fungsi-fungsi untuk meningkatkan kualitas gambar CCTV

In [ ]:
# 4. Fungsi Image Enhancement untuk CCTV Jakarta

def enhance_image(image: np.ndarray) -> np.ndarray:
    """
    Meningkatkan kualitas gambar untuk kondisi CCTV Jakarta yang beragam.
    Optimasi khusus untuk variasi pencahayaan siang/malam dan cuaca.
    
    Args:
        image (np.ndarray): Gambar input dalam format BGR
    
    Returns:
        np.ndarray: Gambar yang telah ditingkatkan kualitasnya
    
    Teknik yang digunakan:
    - CLAHE: Adaptive histogram equalization untuk pencahayaan tidak merata
    - Detail Enhancement: Memperjelas detail untuk kondisi blur/kabur
    """
    # Konversi ke grayscale untuk processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # CLAHE untuk normalisasi pencahayaan (optimal untuk CCTV malam/siang)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_gray = clahe.apply(gray)
    
    # Konversi kembali ke color untuk detail enhancement
    enhanced_color = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)
    
    # Detail enhancement untuk memperjelas teks plat nomor
    enhanced = cv2.detailEnhance(enhanced_color, sigma_s=10, sigma_r=0.15)
    
    return enhanced


def detect_plate_from_vehicle(cropped_img: np.ndarray) -> List[Tuple[int, int, int, int]]:
    """
    Mendeteksi area plat nomor dari gambar kendaraan yang sudah di-crop.
    Menggunakan analisis kontur dengan filter khusus untuk plat Indonesia.
    
    Args:
        cropped_img (np.ndarray): Gambar kendaraan yang sudah di-crop
    
    Returns:
        List[Tuple[int, int, int, int]]: List koordinat (x, y, width, height) kandidat plat
    
    Filter yang diterapkan:
    - Aspect ratio: 2:1 hingga 5:1 (sesuai standar plat Indonesia)
    - Ukuran minimum: width > 50px, height > 15px
    - Edge detection untuk mendeteksi bentuk rectangular
    """
    # Preprocessing untuk edge detection
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)

    # Deteksi kontur untuk kandidat plat nomor
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    plate_candidates = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h

        # Filter berdasarkan karakteristik plat nomor Indonesia
        # Aspect ratio 2-5 dan ukuran minimum sesuai resolusi CCTV
        if (aspect_ratio > 2 and aspect_ratio < 5 and 
            w > 50 and h > 15):
            plate_candidates.append((x, y, w, h))

    return plate_candidates

print("Fungsi image enhancement berhasil didefinisikan!")
print("Siap untuk memproses gambar dengan kualitas optimal")

## Section 5: Main Processing Pipeline
**Tujuan**: Fungsi utama untuk memproses seluruh dataset gambar CCTV Jakarta

In [ ]:
# 5. Main Processing Pipeline untuk Dataset CCTV Jakarta

def process_jakarta_cctv_images(data_folder: str, max_images: int = None) -> dict:
    """
    Memproses semua gambar CCTV Jakarta dalam folder dataset.
    Pipeline lengkap: deteksi kendaraan → lokalisasi plat → OCR → visualisasi.
    
    Args:
        data_folder (str): Path folder yang berisi gambar CCTV
        max_images (int, optional): Batasan jumlah gambar yang diproses
    
    Returns:
        dict: Statistik processing (total gambar, deteksi berhasil, dll)
    
    Pipeline Processing:
    1. Load gambar CCTV
    2. Deteksi kendaraan dengan YOLO
    3. Crop area kendaraan
    4. Deteksi kandidat plat nomor
    5. Enhancement gambar plat
    6. OCR ekstraksi teks
    7. Visualisasi hasil
    """
    
    # Inisialisasi statistik processing
    stats = {
        'total_images': 0,
        'successful_vehicle_detection': 0,
        'successful_plate_detection': 0,
        'total_plates_found': 0,
        'failed_images': []
    }
    
    # Filter file gambar yang didukung
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp')
    image_files = [f for f in os.listdir(data_folder) 
                   if f.lower().endswith(supported_formats)]
    
    if max_images:
        image_files = image_files[:max_images]
    
    print(f"Memulai processing {len(image_files)} gambar CCTV Jakarta...")
    print("=" * 60)
    
    # Processing setiap gambar dalam dataset
    for idx, img_file in enumerate(image_files, 1):
        print(f"\n[{idx}/{len(image_files)}] Processing: {img_file}")
        
        try:
            # Load gambar CCTV
            img_path = os.path.join(data_folder, img_file)
            img = cv2.imread(img_path)
            
            if img is None:
                print(f"Gagal membaca gambar: {img_file}")
                stats['failed_images'].append(img_file)
                continue
            
            stats['total_images'] += 1
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # STEP 1: Deteksi kendaraan dengan YOLO
            print("   Mendeteksi kendaraan...")
            results = model.predict(source=img_path, conf=0.5, save=False, verbose=False)
            
            if not results or not results[0].boxes:
                print("   Tidak ada kendaraan terdeteksi")
                continue
            
            stats['successful_vehicle_detection'] += 1
            vehicle_count = len(results[0].boxes)
            print(f"   {vehicle_count} kendaraan terdeteksi")
            
            # Koleksi hasil untuk gambar ini
            cropped_vehicle_imgs = []
            plate_texts = []
            vehicle_boxes = []
            
            # STEP 2: Process setiap kendaraan yang terdeteksi
            for box_idx, box in enumerate(results[0].boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                vehicle_boxes.append((x1, y1, x2, y2))
                
                # Crop area kendaraan
                cropped_img = img[y1:y2, x1:x2]
                
                # 🔍 STEP 3: Deteksi kandidat plat nomor
                print(f"   Mencari plat pada kendaraan #{box_idx + 1}...")
                plate_candidates = detect_plate_from_vehicle(cropped_img)
                
                if not plate_candidates:
                    print(f"   Tidak ada kandidat plat pada kendaraan #{box_idx + 1}")
                    continue
                
                # 📝 STEP 4: OCR pada setiap kandidat plat
                for plate_idx, (px, py, pw, ph) in enumerate(plate_candidates):
                    # Crop area plat nomor
                    plate_img = cropped_img[py:py+ph, px:px+pw]
                    
                    # Enhancement untuk kondisi CCTV yang beragam
                    plate_img_enhanced = enhance_image(plate_img)
                    
                    # OCR ekstraksi teks
                    try:
                        texts = reader.readtext(plate_img_enhanced, detail=0)
                        if texts:
                            detected_text = ' '.join(texts).strip()
                            if len(detected_text) > 2:  # Filter hasil OCR yang valid
                                plate_texts.append(detected_text)
                                cropped_vehicle_imgs.append(plate_img)
                                print(f"   Plat terdeteksi: '{detected_text}'")
                                stats['total_plates_found'] += 1
                    except Exception as ocr_error:
                        print(f"   OCR error: {ocr_error}")
            
            # 🎨 STEP 5: Visualisasi hasil
            if plate_texts:
                stats['successful_plate_detection'] += 1
                
                # Gambar utama dengan bounding box kendaraan
                img_display = img_rgb.copy()
                for (x1, y1, x2, y2) in vehicle_boxes:
                    cv2.rectangle(img_display, (x1, y1), (x2, y2), (0, 255, 0), 3)
                
                plt.figure(figsize=(15, 10))
                
                # Subplot 1: Gambar asli dengan deteksi kendaraan
                plt.subplot(2, 1, 1)
                plt.imshow(img_display)
                plt.axis('off')
                plt.title(f"{img_file} - {len(vehicle_boxes)} Kendaraan Terdeteksi", 
                         fontsize=14, fontweight='bold')
                
                # Subplot 2: Grid plat nomor yang terdeteksi
                if cropped_vehicle_imgs:
                    plt.subplot(2, 1, 2)
                    cols = min(5, len(cropped_vehicle_imgs))
                    rows = (len(cropped_vehicle_imgs) + cols - 1) // cols
                    
                    for i, (plate_img, text) in enumerate(zip(cropped_vehicle_imgs, plate_texts)):
                        plt.subplot(2, cols, cols + i + 1)
                        plt.imshow(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
                        plt.title(f"{text}", fontsize=10, fontweight='bold')
                        plt.axis('off')
                
                plt.tight_layout()
                plt.show()
                
                print(f"   Sukses! {len(plate_texts)} plat nomor berhasil diidentifikasi")
            else:
                print("   Tidak ada plat nomor yang berhasil diidentifikasi")
                
        except Exception as e:
            print(f"   Error processing {img_file}: {e}")
            stats['failed_images'].append(img_file)
    
    return stats

print("Fungsi main processing pipeline berhasil didefinisikan!")
print("Siap memproses dataset CCTV Jakarta dengan pipeline lengkap")

In [ ]:
# 6. Eksekusi Processing Dataset CCTV Jakarta

# Validasi dan eksekusi processing
if not os.path.exists(data_folder):
    print(f"KESALAHAN: Path dataset tidak ditemukan: {data_folder}")
    print("\nSOLUSI:")
    print("1. Periksa path di Section 2B")
    print("2. Pastikan folder dataset sudah dibuat")
    print("3. Upload gambar CCTV ke folder tersebut")
    print("\nContoh struktur folder:")
    print("   dataset/jakarta_cctv/")
    print("   ├── image1.jpg")
    print("   ├── image2.jpg")
    print("   └── ...")
else:
    # Eksekusi processing dengan progress tracking
    print("MEMULAI SISTEM PENGENALAN PLAT NOMOR JAKARTA")
    print("=" * 60)
    print(f"Dataset: {data_folder}")
    
    # Opsi: Batasi jumlah gambar untuk testing (comment untuk memproses semua)
    # max_images = 5  # Uncomment untuk testing dengan 5 gambar pertama
    max_images = None  # Proses semua gambar
    
    # Jalankan processing pipeline
    try:
        results = process_jakarta_cctv_images(data_folder, max_images)
        
        # 📊 TAMPILKAN STATISTIK HASIL
        print("\n" + "=" * 60)
        print("STATISTIK PROCESSING SELESAI")
        print("=" * 60)
        print(f"Total gambar diproses: {results['total_images']}")
        print(f"Gambar dengan kendaraan terdeteksi: {results['successful_vehicle_detection']}")
        print(f"Gambar dengan plat teridentifikasi: {results['successful_plate_detection']}")
        print(f"Total plat nomor ditemukan: {results['total_plates_found']}")
        
        # Hitung tingkat keberhasilan
        if results['total_images'] > 0:
            vehicle_success_rate = (results['successful_vehicle_detection'] / results['total_images']) * 100
            plate_success_rate = (results['successful_plate_detection'] / results['total_images']) * 100
            
            print(f"\nTINGKAT KEBERHASILAN:")
            print(f"   Deteksi kendaraan: {vehicle_success_rate:.1f}%")
            print(f"   Identifikasi plat: {plate_success_rate:.1f}%")
        
        # Tampilkan gambar yang gagal diproses
        if results['failed_images']:
            print(f"\n  GAMBAR GAGAL DIPROSES ({len(results['failed_images'])}):")  
            for failed_img in results['failed_images']:
                print(f"   {failed_img}")
        
        print("\n PROCESSING DATASET JAKARTA CCTV SELESAI!")
        print("Sistem berhasil menganalisis gambar CCTV dengan variasi cuaca dan pencahayaan")
        
    except Exception as e:
        print(f"KESALAHAN SISTEM: {e}")
        print("\nTROUBLESHOOTING:")
        print("1. Periksa koneksi internet untuk download model")
        print("2. Pastikan format gambar didukung (.jpg, .png, .jpeg)")
        print("3. Cek apakah ada ruang disk yang cukup")
        print("4. Restart kernel jika ada memory error")

---

## Ringkasan Sistem

**Sistem Pengenalan Plat Nomor Jakarta** ini telah dioptimasi untuk:

### **Kondisi CCTV Jakarta**
- Variasi pencahayaan siang/malam
- Kondisi cuaca yang beragam (hujan/cerah)
- Sudut pandang CCTV yang bervariasi
- Lalu lintas padat khas Jakarta

### **Teknologi Unggulan**
- **YOLOv8**: Deteksi kendaraan real-time
- **EasyOCR**: Ekstraksi teks dengan akurasi tinggi
- **CLAHE**: Normalisasi pencahayaan adaptif
- **OpenCV**: Image processing untuk enhancement

### **Hasil Optimal**
- Deteksi multi-kendaraan dalam satu frame
- Filter plat nomor sesuai standar Indonesia
- Visualisasi komprehensif dengan statistik
- Error handling untuk processing yang robust